In [ ]:
#Mounting the google drive
from google.colab import drive
drive.mount('/content/drive')
data_root = '/content/drive/My Drive/MyChatBot'

#Importing Libraries

import json
import string
import random

import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
nltk.download("punkt")
nltk.download("wordnet")

#loading the intents.json dataset
data_file_path = data_root + '/intents.json'
data = json.loads(open(data_file_path).read())

#Creating DataX and DataY
words = []#For bow model/vocabulary for pattern
classes = []#For bow /vocabulary for tags
data_x = [] #for storing each pattern
data_y = [] #for storing each tag corresponding to each pattern in data_x

#Iterating over all the intents
for intent in data["intents"]:
  for pattern in intent["patterns"]:
    tokens = nltk.word_tokenize(pattern) #tokenize each pattern
    words.extend(tokens) #and append tokens to words
    data_x.append(pattern) #appending pattern to data_x
    data_y.append(intent["tag"]), #appending the associated tag to each pattern

  #Adding the tag to the classes if it's not there already
  if intent["tag"] not in classes:
    classes.append(intent["tag"])

#Initializing lemmatizer ot get stem of words
lemmatizer = WordNetLemmatizer()

#Lemmatize all the words in the vocab and convert them to lowercase

#If the words don't appear in punctuation
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]

#sorting
words = sorted(set(words))
classes = sorted(set(classes))

#text to numbers
training = []
out_empty = [0] * len(classes)
# creating the bag of words model
for idx, doc in enumerate(data_x):
  bow = []
  text = lemmatizer. lemmatize(doc.lower())
  for word in words:
    bow.append(1) if word in text else bow.append(0)
  # mark the index of class that the current pattern is associated
  # to
  output_row = list(out_empty)
  output_row[classes.index(data_y[idx])] = 1
  # add the one hot encoded BoW and associated classes to training
  training.append([bow, output_row])
# shuffle the data and convert it to an array
random.shuffle(training)
training = np.array(training, dtype=object)
# split the features and target labels
train_x = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

#Neural network model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]), ), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation = "softmax"))
adam = tf.keras.optimizers.legacy.Adam(learning_rate=0.01, decay=1e-6)
model.compile(loss='categorical_crossentropy',
             optimizer=adam,
             metrics=["accuracy"])
print(model.summary())
model.fit(x=train_x, y=train_y, epochs=150, verbose=1)

#Processing Input
def clean_text(text):
  tokens = nltk.word_tokenize(text)
  tokens = [lemmatizer. lemmatize(word) for word in tokens]
  return tokens

def bag_of_words(text, vocab):
  tokens = clean_text(text)
  bow = [0] * len(vocab)
  for w in tokens:
    for idx, word in enumerate(vocab):
      if word == w:
        bow[idx] = 1
  return np.array(bow)

def pred_class(text, vocab, labels):
  bow = bag_of_words(text, vocab)
  result = model.predict(np.array([bow]))[0] #Extracting probabilities
  thresh = 0.5
  y_pred = [[indx, res] for indx, res in enumerate(result) if res > thresh]
  y_pred.sort(key=lambda x: x[1], reverse=True) #sorting by values of probability in decreasing order
  return_list = []
  for r in y_pred:
    return_list.append(labels[r[0]]) #Contains labels(tags) for highest probability
  return return_list

def get_response(intents_list, intents_json):
  if len(intents_list) == 0:
    result = "Sorry! I don't understand."
  else:
    tag = intents_list[0]
    list_of_intents = intents_json["intents"]
    for i in list_of_intents:
      if i["tag"] == tag:
        result = random.choice(i["responses"])
        break
  return result

#Interacting with ChatBot
print("Press '0' if you don't want to chat with this ChatBot.")
while True:
  message = input("")
  if message == "0":
    break
  intents = pred_class(message, words, classes)
  result = get_response(intents, data)
  print(result)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_57 (Dense)            (None, 128)               7552      
                                                                 
 dropout_38 (Dropout)        (None, 128)               0         
                                                                 
 dense_58 (Dense)            (None, 64)                8256      
                                                                 
 dropout_39 (Dropout)        (None, 64)                0         
                                                                 
 dense_59 (Dense)            (None, 16)                1040      
                                                                 
Total params: 16848 (65.81 KB)
Trainable params: 16848 (

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Epoch 1/150
2/2 [==============================] - 1s 10ms/step - loss: 2.7585 - accuracy: 0.0976
Epoch 2/150
2/2 [==============================] - 0s 8ms/step - loss: 2.6185 - accuracy: 0.1707
Epoch 3/150
2/2 [==============================] - 0s 10ms/step - loss: 2.4621 - accuracy: 0.2683
Epoch 4/150
2/2 [==============================] - 0s 9ms/step - loss: 2.3439 - accuracy: 0.2439
Epoch 5/150
2/2 [==============================] - 0s 9ms/step - loss: 2.1574 - accuracy: 0.2683
Epoch 6/150
2/2 [==============================] - 0s 9ms/step - loss: 1.8926 - accuracy: 0.4146
Epoch 7/150
2/2 [==============================] - 0s 9ms/step - loss: 1.9911 - accuracy: 0.3415
Epoch 8/150
2/2 [==============================] - 0s 9ms/step - loss: 1.7481 - accuracy: 0.3902
Epoch 9/150
2/2 [==============================] - 0s 9ms/step - loss: 1.6957 - accuracy: 0.4878
Epoch 10/150
2/2 [==============================] - 0s 15ms/step - loss: 1.4360 - accuracy: 0.5610
Epoch 11/150
2/2 [========